<a href="https://colab.research.google.com/github/yajima-yasutoshi/Model/blob/main/20250806/%E7%A2%BA%E8%AA%8D%E3%83%86%E3%82%B9%E3%83%88.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#確認テスト（20250806）

##準備


In [ ]:
%%capture
# python-mip: 数理最適化モデリングライブラリ
!pip install mip

# japanize-matplotlib: Matplotlibでの日本語表示を可能にするライブラリ
!pip install japanize-matplotlib

# 必要なライブラリのインポート
import mip
import numpy as np
import matplotlib.pyplot as plt
import japanize_matplotlib
import random
from scipy.spatial.distance import cdist
from itertools import product

-----
##問題1

ある企業が、複数の顧客にサービスを提供するために、いくつかの候補地の中から倉庫（施設）を建設することを計画しています。

* **施設候補地:** 3ヶ所 (F1, F2, F3)。各候補地には倉庫を開設するための固定費用があります。
* **施設候補地の固定費用 ($f_i$):**
    * F1: 固定費用 150
    * F2: 固定費用 100
    * F3: 固定費用 130
* **顧客:** 4ヶ所 (C1, C2, C3, C4)
* **輸送費用 ($c_{ij}$): 施設 $i$ から顧客 $j$ へ輸送する費用:**

|       | C1     | C2       | C3       | C4  |
| :---- | :------: | :------: | :------: | :------: |
| F1  |    20    |    10    |    50    |    30    |
| F2  |    60    |    30    |    20    |    10    |
| F3  |    10    |    40    |    15    |    50    |

どの候補地に施設を開設し、
各顧客へどの開設済みの施設からサービスを提供すれば固定費用と輸送費用の合計が最小になるかを求め、最適値を解答せよ。


### 問題1-1

開設した施設からサービスを提供できる顧客数を最大3顧客までとした場合、 どの候補地に施設を開設し各顧客へどの開設済みの施設からサービスを提供すれば固定費用と輸送費用の合計が最小になるかを求め、最適値を解答せよ。

-----

##問題2

以下のように6都市の座標が与えられたとする。
デポ（出発・帰着）を都市0とし、
6都市全てを巡回する最短経路の総移動距離を求めよ。
移動距離は都市の座標から計算されるユークリッド距離とする。

解答は小点以下第二位を四捨五入して、第一位までを解答する。

###データ

  * **変更後の都市座標**:
      * 都市0: (0, 0)
      * 都市1: (1, 5)
      * 都市2: (4, 3)
      * 都市3: (2, 1)
      * 都市4: (5, 5)
      * 都市5: (2, 5)

#### ユークリッド距離の計算
一般的に、都市Aと都市Bの座標が以下の場合
* 都市A： $(x_A,y_A)$
* 都市B： $(x_B,y_B)$

ユークリッド距離は
$$
\sqrt{(x_A - x_B)^2 + (y_A - y_B)^2}
$$
と計算される。



###問題2-1

* 都市1は都市3の前
* 都市3は都市5の前

に訪問しなくてはならないとする。
なお、都市1と都市3の間や
都市3と都市5の間に別の都市が入っても構わない。
この場合の最適値を求めよ。
解答は小点以下第二位を四捨五入して、第一位までを解答する。


-----

##問題3

ある企業が2つの工場（F1, F2）で製品を生産し、2つの中間倉庫（W1, W2）を経由して、3つの主要な顧客（C1, C2, C3）へ製品を供給する計画を立ている。
各工場には生産上限（供給量）、各顧客には需要量がある。
工場から倉庫へ、倉庫から顧客へ、そして一部の工場から顧客への直接輸送ルートも存在し、それぞれ輸送コストと輸送容量が設定されている。

* **ノードとバランス ($b_k$):**
    * F1 (工場1): 供給量が100 ユニット
    * F2 (工場2): 供給量が80 ユニット
    * W1 (倉庫1): 中継点
    * W2 (倉庫2): 中継点
    * C1 (顧客1): 需要量が85 ユニット (つまり-85ユニットの供給量)
    * C2 (顧客2): 需要量が50 ユニット (つまり-50ユニットの供給量)
    * C3 (顧客3): 需要量が60 ユニット (つまり-60ユニットの供給量)
  
  なお、総供給量 = $100+80=180$、総需要量 = $85+50+60=195$ となり、需要と共有は均衡していない。

* **アーク (始点, 終点, 単位費用, 容量):** 以下の示すようなアークを考える。
    * (F1, W1, 2, 70), (F1, W2, 3, 50)
    * (F2, W1, 4, 60), (F2, W2, 1, 50)
    * (W1, C1, 3, 40), (W1, C2, 5, 30), (W1, C3, 4, 40)
    * (W2, C1, 2, 50), (W2, C2, 1, 40), (W2, C3, 3, 30)
    * (F1, C1, 10, 20)  // F1からC1への直接輸送
    * (F2, C3, 12, 20)  // F2からC3への直接輸送

総需要(195)が総供給(180)を上回り、15単位の需要が満たせない状況である。満たされなかった需要1単位あたり**20**のペナルティ費用が発生する場合、実際の総輸送費用とペナルティ費用の合計を最小化せよ。その最小費用を解答せよ。



-----

##問題4
ある会社が、製品を特殊なコンテナ（ビン）に詰めて輸送する計画を立てている。各製品（アイテム）はそれぞれ異なる重量と容積を持つ。全ての製品を輸送するために必要な最小のコンテナ数を求めよ。

**データ:**

  * **コンテナの容量:**
   * 最大積載重量 $C\_{weight}$: 17 kg
   * 最大容積 $C\_{volume}$: 11 m³
  * **製品（アイテム）のリスト（重量, 容積）:**
      * アイテム0: (5 kg, 4 m³)
      * アイテム1: (8 kg, 6 m³)
      * アイテム2: (3 kg, 2 m³)
      * アイテム3: (10 kg, 5 m³)
      * アイテム4: (6 kg, 8 m³)
      * アイテム5: (7 kg, 4 m³)
      * アイテム6: (9 kg, 4 m³)
      * アイテム7: (7 kg, 7 m³)


-----
##問題5

あるネットワークにおいて、ソースノード(S)から
シンクノード(T)へのフローを考える。
アーク容量に加え、一部のノードにはそこを通過するフローに
上限がある。
これらの制約下で、SからTへの最大フローを求めよ。

**問題設定:**

  * **ノード:** S, A, B, C, T
  * **アーク容量:**
      * (S, A): 20
      * (S, B): 15
      * (A, C): 10
      * (B, A): 5
      * (B, C): 12
      * (C, T): 18
      * (A, T): 2
      * (B, T): 2

  * **ノード容量:**
      * ノードA: 18
      * ノードB: 9


-----

##問題6

ある旅行者が、スタート地点（ノード0）から出発し、制限時間 $T_{max}$ 内にいくつかの観光地（ノード1から5）を巡り、最終的にゴール地点（ノード6）に到達する計画を立てています。各観光地には訪問することで得られるスコアがあります。

* **ノード:**
    * ノード0: スタート地点, 座標(0,0), スコア $p_0=0$
    * ノード1: 観光地A, 座標(1,3), スコア $p_1=10$
    * ノード2: 観光地B, 座標(3,4), スコア $p_2=40$
    * ノード3: 観光地C, 座標(2,3.5), スコア $p_3=10$
    * ノード4: 観光地D, 座標(3,3), スコア $p_2=20$
    * ノード5: 観光地B, 座標(2,4), スコア $p_2=25$
    * ノード6: ゴール地点, 座標(4,1), スコア $p_3=0$
* **移動時間 ($t_{ij}$):** ノード間のユークリッド距離を移動時間とします。
* **総移動時間上限 ($T_{max}$):** 8.2単位時間。

2つの独立した経路 (2人の旅行者、または2台の車両) でスコア地点を分担して訪問できるとします。各経路の総移動時間上限はそれぞれ $T_{max}$ です。
スコア地点は一度訪問されればスコアが得られ、複数の経路で同じスコア地点を訪問してもスコアは重複して加算されません。全体の獲得総スコアの最大値を求めなさい。
